## Introduction

We'll learn another way pandas makes working with data easier. It has many built-in methods and functions for common exploration and analysis tasks.

__Dataset__: Fortune Global 500 list [f500.csv]

__Source__: https://data.world/chasewillden/fortune-500-companies-2017

Here is a data dictionary for some of the columns in the CSV:

- ```company```: Name of the company.
- ```rank```: Global 500 rank for the company.
- ```revenues```: Company's total revenue for the fiscal year, in millions of dollars (USD).
- ```revenue_change```: Percentage change in revenue between the current and prior fiscal year.
- ```profits```: Net income for the fiscal year, in millions of dollars (USD).
- ```ceo```: Company's Chief Executive Officer.
- ```industry```: Industry in which the company operates.
- ```sector```: Sector in which the company operates.
- ```previous_rank```: Global 500 rank for the company for the prior year.
- ```country```: Country in which the company is headquartered.
- ```hq_location```: City and Country, (or City and State for the USA) where the company is headquarted.
- ```employees```: Total employees (full-time equivalent, if available) at fiscal year-end.

In [1]:
import pandas as pd

f500 = pd.read_csv('data/f500.csv', index_col=0)
f500_head = f500.head(10)
f500.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, Walmart to AutoNation
Data columns (total 16 columns):
rank                        500 non-null int64
revenues                    500 non-null int64
revenue_change              498 non-null float64
profits                     499 non-null float64
assets                      500 non-null int64
profit_change               436 non-null float64
ceo                         500 non-null object
industry                    500 non-null object
sector                      500 non-null object
previous_rank               500 non-null int64
country                     500 non-null object
hq_location                 500 non-null object
website                     500 non-null object
years_on_global_500_list    500 non-null int64
employees                   500 non-null int64
total_stockholder_equity    500 non-null int64
dtypes: float64(3), int64(7), object(6)
memory usage: 52.7+ KB


### Vectorized Operations

Just like with NumPy, we can use any of the standard Python numeric operators with series, including:
- ```series_a + series_b``` - Addition
- ```series_a - series_b``` - Subtraction
- ```series_a * series_b``` - Multiplication (this is unrelated to the multiplications used in linear algebra).
- ```series_a / series_b``` - Division

In [2]:
rank_change = f500.loc[:, "previous_rank"] - f500.loc[:, "rank"]

### Series Data Exploration Methods

Like NumPy, pandas supports many descriptive stats methods that can help us answer these questions. Here are a few of the most useful ones (with links to documentation):
- ```Series.max()```
- ```Series.min()```
- ```Series.mean()```
- ```Series.median()```
- ```Series.mode()```
- ```Series.sum()```

In [3]:
rank_change_max = rank_change.max()
rank_change_min = rank_change.min()

### Series Describe Method

We used the Series.max() and Series.min() methods to figure out the biggest increase and decrease in rank:
- Biggest increase in rank: 226
- Biggest decrease in rank: -500

However, according to the data dictionary, this list should only rank companies on a scale of 1 to 500. Even if the company ranked 1st in the previous year moved to 500th this year, the rank change calculated would be -499. This indicates that there is incorrect data in either the ```rank``` column or ```previous_rank``` column.

We'll learn another method that can help us more quickly investigate this issue - the ```Series.describe()``` method. This method tells us how many non-null values are contained in the series, along with the mean, minimum, maximum, and other statistics.

The first statistic, __count__, is the same as for numeric columns, showing us the number of non-null values. The other three statistics are new:
- ```unique```: Number of unique values in the series.
- ```top```: Most common value in the series.
- ```freq```: Frequency of the most common value.

In [4]:
rank = f500["rank"]
rank_desc = rank.describe()

prev_rank = f500["previous_rank"]
prev_rank_desc = prev_rank.describe()

In [5]:
prev_rank_desc

count    500.000000
mean     222.134000
std      146.941961
min        0.000000
25%       92.750000
50%      219.500000
75%      347.250000
max      500.000000
Name: previous_rank, dtype: float64

### Method Chaining

We notice that the minimum rank is 0, which is odd! To investigate the possible cause of this issue, let's confirm the number of 0 values that appear in the previous_rank column.

We can skip some of the intermediate code assignments. This is called __method chaining__ — a way to combine multiple methods together in a single line.
- When writing code, always assess whether method chaining will make your code harder to read. If it does, it's always preferable to break the code into more than one line.

In [6]:
# Count the number of zeros in our previous_rank column
zero_previous_rank = f500["previous_rank"].value_counts().loc[0]

### Dataframe Exploration Methods

We confirmed that 33 companies in the dataframe have a value of 0 in the previous_rank column. Given that multiple companies have a 0 rank, we might conclude that these companies didn't have a rank at all for the previous year. It would make more sense for us to replace these values with a null value instead.

Because series and dataframes are two distinct objects, they have their own unique methods. However, there are many times where both series and dataframe objects have a method of the same name that behaves in similar ways. Below are some examples:
- ```Series.max()``` and ```DataFrame.max()```
- ```Series.min()``` and ```DataFrame.min()```
- ```Series.mean()``` and ```DataFrame.mean()```
- ```Series.median()``` and ```DataFrame.median()```
- ```Series.mode()``` and ```DataFrame.mode()```
- ```Series.sum()``` and ```DataFrame.sum()```

Unlike their series counterparts, dataframe methods require an _axis parameter_ so we know which axis to calculate across. While you can use integers to refer to the first and second axis, pandas dataframe methods also accept the strings ```"index"``` and ```"columns"``` for the axis parameter

In [7]:
# find the maximum value for only the numeric columns from f500
max_f500 = f500.max(numeric_only=True)
max_f500

rank                            500.0
revenues                     485873.0
revenue_change                  442.3
profits                       45687.0
assets                      3473238.0
profit_change                  8909.5
previous_rank                   500.0
years_on_global_500_list         23.0
employees                   2300000.0
total_stockholder_equity     301893.0
dtype: float64

### Dataframe Describe Method

Based on the column descriptions, the maximum for each of these columns seems reasonable. Like series objects, dataframe objects also have a ```DataFrame.describe()``` method that we can use to explore the dataframe more quickly.

One difference is that we need to manually specify if you want to see the statistics for the non-numeric columns. By default, ```DataFrame.describe()``` will return statistics for only numeric columns. If we wanted to get just the object columns, we need to use the ```include=['O']``` parameter

Whereas the ```Series.describe()``` method returns a series object, the ```DataFrame.describe()``` method returns a dataframe object

In [8]:
f500_desc = f500.describe()
f500_desc

,rank,revenues,revenue_change,profits,assets,profit_change,previous_rank,years_on_global_500_list,employees,total_stockholder_equity
count,500.000000,500.000000,498.000000,499.000000,5.000000e+02,436.000000,500.000000,500.000000,5.000000e+02,500.000000
mean,250.500000,55416.358000,4.538353,3055.203206,2.436323e+05,24.152752,222.134000,15.036000,1.339983e+05,30628.076000
std,144.481833,45725.478963,28.549067,5171.981071,4.851937e+05,437.509566,146.941961,7.932752,1.700878e+05,43642.576833
min,1.000000,21609.000000,-67.300000,-13038.000000,3.717000e+03,-793.700000,0.000000,1.000000,3.280000e+02,-59909.000000
25%,125.750000,29003.000000,-5.900000,556.950000,3.658850e+04,-22.775000,92.750000,7.000000,4.293250e+04,7553.750000
50%,250.500000,40236.000000,0.550000,1761.600000,7.326150e+04,-0.350000,219.500000,17.000000,9.291050e+04,15809.500000
75%,375.250000,63926.750000,6.975000,3954.000000,1.805640e+05,17.700000,347.250000,23.000000,1.689172e+05,37828.500000
max,500.000000,485873.000000,442.300000,45687.000000,3.473238e+06,8909.500000,500.000000,23.000000,2.300000e+06,301893.000000


### Assignment with pandas

After reviewing the descriptive statistics for the numeric columns in f500, we can conclude that no values look unusual besides the 0 values in the previous_rank column.

We'll learn how to do two things so we can correct these values:
- __Perform assignment in pandas.__
- Use boolean indexing in pandas.

Just like in NumPy, the same techniques that we use to select data could be used for assignment. When we selected a whole column by label and used assignment, we assigned the value to every item in that column.

By providing labels for both axes, we can assign them to a single value within our dataframe.

In [9]:
f500.loc["Dow Chemical","ceo"] = "Jim Fitterling"

f500.loc["Dow Chemical", "ceo"]

'Jim Fitterling'

### Using Boolean Indexing with pandas Objects

While it's helpful to be able to replace specific values when we know the row label ahead of time, this can be cumbersome when we need to replace many values. Instead, we can use __boolean indexing__ to change all rows that meet the same criteria, just like we did with NumPy.

Next, let's use boolean indexing to identify companies belonging to the "Motor Vehicles and Parts" industry in our Fortune 500 dataset.

In [10]:
motor_bool = f500["industry"] == "Motor Vehicles and Parts"

motor_countries = f500.loc[motor_bool, "country"] # one specific column

motor_countries

company
Toyota Motor                                 Japan
Volkswagen                                 Germany
Daimler                                    Germany
General Motors                                 USA
Ford Motor                                     USA
Honda Motor                                  Japan
SAIC Motor                                   China
Nissan Motor                                 Japan
BMW Group                                  Germany
Dongfeng Motor                               China
Robert Bosch                               Germany
Hyundai Motor                          South Korea
China FAW Group                              China
Beijing Automotive Group                     China
Peugeot                                     France
Renault                                     France
Kia Motors                             South Korea
Continental                                Germany
Denso                                        Japan
Guangzhou Automobile In

### Using Boolean Arrays to Assign Values

We now have all the knowledge we need to fix the 0 values in the previous_rank column:
- Perform assignment in pandas.
- Use boolean indexing in pandas.

We can remove the intermediate step of creating a boolean series, and combine everything into one line. This is the most common way to write pandas code to perform assignment using boolean arrays

Now we can follow this pattern to replace the values in the previous_rank column. We'll replace these values with ```np.nan```. Just like in NumPy, ```np.nan``` is used in pandas to represent values that can't be represented numerically, most commonly missing values.

To make comparing the values in this column before and after our operation easier, we've added the following line of code:
```
prev_rank_before = f500["previous_rank"].value_counts(dropna=False).head()
```
This uses ```Series.value_counts()``` and ```Series.head()``` to display the 5 most common values in the ```previous_rank``` column, but adds an additional ```dropna=False``` parameter, which stops the ```Series.value_counts()``` method from excluding null values when it makes its calculation.

In [11]:
import numpy as np

# Pandas series for the prev_rank for comparison
prev_rank_before = f500["previous_rank"].value_counts(dropna=False).head()

# Changing the rank 0 to NAN
f500.loc[f500["previous_rank"] == 0, "previous_rank"] = np.nan
# The After series
prev_rank_after = f500["previous_rank"].value_counts(dropna=False).head()

### Creating New Columns

You may have noticed that after we assigned NaN values, the previous_rank column changed dtype.

The index of the series that Series.value_counts() produces now shows us floats like 471.0 instead of integers.
- Pandas uses the NumPy integer dtype, which does not support NaN values.
- Pandas inherits this behavior, and in instances where you try and assign a NaN value to an integer column, pandas will silently convert that column to a float dtype.

Now that we've corrected the data, let's create the rank_change series again. This time, we'll add it to our f500 dataframe as a new column.

In [12]:
f500["rank_change"] = f500["previous_rank"] - f500["rank"]

rank_change_desc = f500["rank_change"].describe()

## Challenge: Top Performers by Country

In this challenge, we'll calculate a specific statistic or attribute of each of the three most common countries from our f500 dataframe.

Tasks:
- Create a series, industry_usa, containing counts of the two most common values in the industry column for companies headquartered in the USA.
- Create a series, sector_china, containing counts of the three most common values in the sector column for companies headquartered in the China.
- Create a float object, mean_employees_japan, containing the mean (average) number of employees for companies headquartered in Japan.

In [13]:
top_3_countries = f500["country"].value_counts().head(3)

# counts of the two most common values in the industry column for companies headquartered in the USA
industry_usa = f500.loc[f500["country"] == "USA", "industry"].value_counts().head(2)

# counts of the three most common values in the sector column for companies headquartered in the China.
sector_china = f500.loc[f500["country"] == "China", "sector"].value_counts().head(3)

# Not literal Mean ;), if only!
# containing the mean (average) number of employees for companies headquartered in Japan.
mean_employees_japan = f500.loc[f500["country"] == "Japan", "employees"].mean()

## Conclusion:

### Fundamentals
- How to select data from pandas objects using boolean arrays.
- How to assign data using labels and boolean arrays.
- How to create new rows and columns in pandas.
- Many new methods to make data analysis easier in pandas.

Now we will continue with some advance pandas concepts

## Exploring data with pandas: Intermediate 

We'll continue working with the 2017 Fortune Global 500 dataset as we learn more advanced selection and exploration techniques. As a reminder, the data dictionary for the main columns in the f500.csv file is below:
- company: Name of the company.
- rank: Global 500 rank for the company.
- revenues: Company's total revenue for the fiscal year, in millions of dollars (USD).
- revenue_change: Percentage change in revenue between the current and prior fiscal year.
- profits: Net income for the fiscal year, in millions of dollars (USD).
- sector: Sector in which the company operates.
- previous_rank: Global 500 rank for the company for the prior year.
- country: Country in which the company is headquartered.
- hq_location: City and country, (or city and state for the USA) where the company is headquartered.
- employees: Total employees (full-time equivalent, if available) at fiscal year-end.

In [14]:
import pandas as pd
import numpy as np

# Read the data into a pandas dataframe
f500 = pd.read_csv('data/f500.csv', index_col=0)
f500.index.name = None

# Replace the 0 values in the 'previous_rank' column with NaN
f500.loc[f500["previous_rank"] == 0, "previous_rank"] = np.nan

f500_selection = f500[["rank", "revenues", "revenue_change"]].head() # select the top 5 rows from the resp columns

### Reading CSV files with pandas
```
f500 = pd.read_csv("f500.csv", index_col=0)
f500.index.name = None
```

When we compared the files, the index axis labels are actaully the values of the first column in the dataset. ```company```

The ```index_col``` parameter is an optional argument and should specify which column to use as the row labels for the dataframe. When we used a value of ```0```, we specified that we wanted to use the first column as the row labels.

In [15]:
f500 = pd.read_csv('data/f500.csv') # without the index_col parameter
f500.loc[f500["previous_rank"] == 0, "previous_rank"] = np.nan

### Using iloc to select by integer position



We read our CSV file into pandas again. However, this time, we didn't use the ```index_col``` parameter

There are two differences with this approach:
- The company column is now included as a regular column, instead of being used for the index.
- The index labels are now integers starting from 0.

In some scenarios, using labels to make selections makes things easier — in others though, it makes things harder.
- Just like in NumPy, we can also use integer positions to select data using ```Dataframe.iloc[]``` and ```Series.iloc[]```. It's easy to get ```loc[]``` and ```iloc[]``` confused at first, but the easiest way is to remember the first letter of each method:
    - __l__oc: __l__abel based selection
    - __i__loc: __integer__ position based selection
- Using ```iloc[]``` is almost identical to indexing with NumPy, with integer positions starting at 0 like ndarrays and Python lists.

The full syntax for ```DataFrame.iloc[]```, in pseudocode, is:

```df.iloc[row_index, column_index]```

In [16]:
fifth_row = f500.iloc[4] # 5th row of the f500 dataframe

company_value = f500.iloc[0, 0] # selct the first row of the first column 'company'